# RNN

RNN 存在两个特殊的问题, 梯度爆炸(exploding gradient problem), 梯度离散. 

我们先来看一下rnn的梯度公式

![image](./../../assets/23.png)

在这里我们可以看到, 最终计算的$W$是以一个乘机计算出来的, 当我们的训练数据极其庞大的时候, 如果w稍大于1, 那么w最终就会变得无穷大. 这就是梯度爆炸.同时, 如果w略小于1, 那么经过无数次迭代以后, 梯度就会变得无穷小. 这也就成为梯度离散.


解决梯度爆炸: 如图所示, 这种情况很可能出现梯度爆炸, 比如我们尝试向左进行梯度更新, 但是有个断崖使得error忽然增大. 导致error增大到一个非常大的值, 此时需要设定一个机制. 

![image](./../../assets/24.jpeg)


我们可以设定这个机制, 我们知道W.grad保存了我们的grid值, 我们可以检查一下, 如果大于一个值, 那么就直接设定一个阈值. 我们用梯度的tensor除以梯度的模. 然后在这之上添加一个阈值, 比如15. 这样梯度的方向还是保持不变. 这就是我们这一步走的小一点. *注意我是对梯度grid进行取模, 不是weight*

![image](./../../assets/25.png)

```python
loss = criterion(output, y)
model.zero_grad()
loss.backward()
# 然后我们对grid进行clip, 我们把tensor传入, 我们将grid clip到一个小于10的范围
for p in model.parameters():
    print(p.grad.norm())  # 这里我们查看一下梯度, 我们可以尝试简单的搜索
    torch.nn.utils.clip_grad_norm_(p, 10)  # 我们可以设置一个if, 如果发现爆炸的情况直接使用这段代码, 对于每一个p进行clip
optimizer.step()
```

梯度离散也可能出现在cnn中. 在反向传播中, 我们的k进行反向传播数据的时候会累积误差.

这个误差会随着网络的结构更加复杂而导致传播误差越来越大. 层数越多, 反向传播到最初一层的误差就会越大. 当我们训练的时候, grid在后面几层反向计算的时候比较大, 但是到了前面几层, grid的计算就非常非常的小. 所以即使网络堆叠的再多, 也未必会完整的得到训练. 这个问题到今天依旧存在于cnn中.

对于rnn, 这种问题, 依旧存在于梯度离散问题, 解决这个问题就是LSTM.

## LSTM

long short-term memory networks. 我们的memory节点在NLP设计之初是用来保存语境的, 这样到了后面的节点我们依旧可以懂得我们之前说的是什么意思, 经过长时间的计算, 我们的memory节点可能只能记住最后几个词. LSTM还解决了长度的问题.

![image](./../../assets/26.png)

我们构建3个门, 我们有选择的让输入, 输出, h和h的梯度计算进行在当前节点上. 这里的门就是sigmoid函数, 我们就可以启动控制输出的方式. 图中, $\sigma$就是sigmoid函数.

需要注意, 在定义网络的时候, 我们定义h的大小同时反映了C的大小, 因为两个矩阵需要做运算, 因此大小是相同的.

$out, (ht,ct) = lstm(x,[ht_1,ct_1])$

out也是所有层所有sequence的所有输出, 注意这里只有h不需要c, ct和ht都是结果
- x: [seq,b,vec]
- h/c: [layers,b,h]
- out: [seq,b,h]

In [1]:
import torch
from torch import nn
import numpy as np
from matplotlib import pyplot as plt


In [2]:
lstm = nn.LSTM(100, 20, 4)  # 20个hidden, 100维向量
print(lstm)

x = torch.randn(10, 3, 100)
out, (h, c) = lstm(x)  # lstm(x,[h,c]) 如果不指定就直接使用0
print(out.shape, h.shape, c.shape)
# H: [4,3,20] 一个4层, 每一层3个单词, 每个单词20个向量用来记忆.


LSTM(100, 20, num_layers=4)
torch.Size([10, 3, 20]) torch.Size([4, 3, 20]) torch.Size([4, 3, 20])


第二种方式, 使用cell

相比于直接定义lstm,这种更灵活. 在定义的时候都是一样的, 但是前向传播不太一样

$ht_1,ct_1 = lstmcell(xt,[ht_0,ct_0])$

In [14]:
## 第二种方式, 使用cell
cell = nn.LSTMCell(100, 20)
h = torch.zeros(3, 20)
c = torch.zeros(3, 20)
x = torch.randn(3, 100)
for xt in x:
    h, c = cell(x, [h, c])
print(h.shape, c.shape)


torch.Size([3, 20]) torch.Size([3, 20])


In [17]:
# 定义多层
x = torch.randn(10, 3, 100)

cell1 = nn.LSTMCell(100, 30)
cell2 = nn.LSTMCell(30, 20)
h1 = torch.zeros(3, 30)
c1 = torch.zeros(3, 30)
h2 = torch.zeros(3, 20)
c2 = torch.zeros(3, 20)
for xt in x:
    h1, c1 = cell1(xt, [h1, c1])
    h2, c2 = cell2(h1, [h2, c2])

print(h2.shape, c2.shape)


torch.Size([3, 20]) torch.Size([3, 20])
